In [1]:
import gene_expression_helper as helper
import alignment_helper as algn
import os
import SimpleITK as sitk

#
# Example python code to demostrate for an image-series how to
# download metadata, downsampled images, registration parameter
# and create registered volumes
#
# Find the image series you want from the Allen Brain Atlas portal
# Take a note of the dataset id
#

# Copy the dataset id here
dataset_id = 100054762
product_id = 3

# Specify output directory
output_directory = os.path.join('temp','%d' % dataset_id )
os.makedirs( output_directory, exist_ok = True )

# Specify downsample factor
downsample_factor = 3

# Location of atlas directory
atlas_directory = r'C:\Users\lydia\OneDrive\Work\DigitalAssets\atlas_models'

In [2]:
#
# Download metadata
# - The output json file contains all the information an experiment page
# - Example: http://developingmouse.brain-map.org/experiment/show/100076465
#
metadata_json = os.path.join(output_directory, "metadata_%d.json" % (dataset_id) )
helper.download_metadata( dataset_id, metadata_json, product_id )

In [3]:
#
# Download downsampled images
#
image_directory = os.path.join( output_directory, 'section_images' )
os.makedirs( image_directory, exist_ok = True )
helper.download_images(dataset_id,image_directory,downsample_factor)

In [4]:
#
# Download alignment parameters
# - The output json file contains all the alignment parameters 
# - computed as part of the informatics processing pipeline
#
alignment_json = os.path.join(output_directory, "alignment_%d.json" % (dataset_id) )
helper.download_alignment_parameters( dataset_id, alignment_json, product_id )

In [5]:
#
# Use the alignment parameters to create reconstructed and registered volumes
#
xform = algn.read_json( alignment_json)

# generate reconstructed volume
output = algn.initialize_output( xform, atlas_directory, downsample_factor )
algn.populate_output( xform, image_directory, downsample_factor, output, algn.BLUECHANNEL )

# read atlas
atlas = algn.read_atlas( atlas_directory, xform['age'], xform['plane_of_section'])

# generate registered volumes
algn.resample_to_atlas( xform, atlas, output )

# write data to output
resampled_directory = os.path.join( output_directory, 'resampled')
os.makedirs( resampled_directory, exist_ok = True )
algn.write_volumes( output, resampled_directory )

#
# output files
#
# volume: a 3D reconstruction from the downsampled images
# mask: non-zero value indiciate which section has valid image data
# resampled_volume: reconstructed volume registered to age-plane matched atlas
# resampled_mask: non-zero value indicate which section has valid image data
# resampled_atlas: age-plane matched atlas registered to the reconstruction ISH volume
#